*** SKIN DISEASE USING CNN ***

Number of Classes - 23 

DATASET

1.   Training Dataset - 15557 Files in JPG Format
2.   Testing Dataset - 4002 Files in JPG Format



In [6]:
#Import necessary libraries and modules
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16
from keras.preprocessing.image import ImageDataGenerator
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [7]:
#Define the input image dimensions and the number of classes in your dataset
img_height = 224
img_width = 224
num_classes = 23

In [8]:
#Create a data generator object to preprocess and augment your dataset
data_generator = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        validation_split=0.2)

In [9]:
#Drive Access
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Use the data generator object to load and split your dataset into training and validation sets
train_generator = data_generator.flow_from_directory(
        '/content/drive/MyDrive/FINAL YEAR PROJECT/train',
        target_size=(img_height, img_width),
        batch_size=32,
        class_mode='categorical',
        subset='training') # set as training data

validation_generator = data_generator.flow_from_directory(
        '/content/drive/MyDrive/FINAL YEAR PROJECT/train',
        target_size=(img_height, img_width),
        batch_size=32,
        class_mode='categorical',
        subset='validation')

Found 12463 images belonging to 23 classes.
Found 3106 images belonging to 23 classes.


In [10]:
#Define a CNN model architecture using the tf.keras.Sequential API
model = tf.keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

In [11]:
#Compile the model with appropriate loss and optimizer functions
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])

In [ ]:
#Train the model on your dataset using the fit_generator method
history = model.fit_generator(
        train_generator,
        steps_per_epoch=train_generator.samples//train_generator.batch_size,
        epochs=10,
        validation_data=validation_generator,
        validation_steps=validation_generator.samples//validation_generator.batch_size)

<ipython-input-8-428efbb955d4>:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/10
389/389 [==============================] - 2787s 7s/step - loss: 3.0546 - accuracy: 0.1417 - val_loss: 2.9388 - val_accuracy: 0.1186
Epoch 2/10
389/389 [==============================] - 1499s 4s/step - loss: 2.7614 - accuracy: 0.1789 - val_loss: 2.8676 - val_accuracy: 0.1698
Epoch 3/10
389/389 [==============================] - 1487s 4s/step - loss: 2.6651 - accuracy: 0.2096 - val_loss: 2.8699 - val_accuracy: 0.1827
Epoch 4/10
389/389 [==============================] - 1483s 4s/step - loss: 2.5933 - accuracy: 0.2292 - val_loss: 2.8229 - val_accuracy: 0.1814
Epoch 5/10
389/389 [==============================] - 1494s 4s/step - loss: 2.5294 - accuracy: 0.2492 - val_loss: 2.8333 - val_accuracy: 0.1885
Epoch 6/10
389/389 [==============================] - 1489s 4s/step - loss: 2.4814 - accuracy: 0.2593 - val_loss: 2.8358 - val_accuracy: 0.1840
Epoch 7/10
389/389 [==============================] - 1520s 4s/step - loss: 2.4225 - accuracy: 0.2723 - val_loss: 2.8567 - val_accuracy:

In [12]:
# Evaluate the model on the test set
test_generator = data_generator.flow_from_directory(
        '/content/drive/MyDrive/FINAL YEAR PROJECT/test',
        target_size=(img_height, img_width),
        batch_size=32,
        class_mode='categorical')

scores = model.evaluate(test_generator)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Found 4002 images belonging to 23 classes.
126/126 [==============================] - 824s 7s/step - loss: 3.1559 - accuracy: 0.0315
Test loss: 3.155900478363037
Test accuracy: 0.03148425742983818


In [13]:
# Load pre-trained CNN model
model = VGG16(weights='imagenet', include_top=False)

# Extract features from images
datagen = ImageDataGenerator(rescale=1./255)
train_generator = datagen.flow_from_directory(
        '/content/drive/MyDrive/FINAL YEAR PROJECT/train',
        target_size=(224, 224),
        batch_size=32,
        class_mode=None,
        shuffle=False)
train_features = model.predict(train_generator)
test_generator = datagen.flow_from_directory(
        '/content/drive/MyDrive/FINAL YEAR PROJECT/test',
        target_size=(224, 224),
        batch_size=32,
        class_mode=None,
        shuffle=False)
test_features = model.predict(test_generator)

# Flatten features
train_features = train_features.reshape(train_features.shape[0], -1)
test_features = test_features.reshape(test_features.shape[0], -1)

# Train SVC model
svc = SVC(kernel='linear')
svc.fit(train_features, train_generator.classes)

# Test SVC model
y_pred = svc.predict(test_features)
accuracy = accuracy_score(test_generator.classes, y_pred)
print("Accuracy:", accuracy)

58889256/58889256 [==============================] - 0s 0us/step
Found 15569 images belonging to 23 classes.
487/487 [==============================] - 8915s 18s/step
Found 4002 images belonging to 23 classes.
126/126 [==============================] - 2267s 18s/step
Accuracy: 0.392303848075962
